<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #12: Analyse Zooniverse classifications</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Aug 24th, 2021</h5>

### Step 0: Import Python packages and initiate koster database

In [ ]:
# Save your user name, password and Zooniverse project number.
import getpass
zoo_user = getpass.getpass('Enter your Zooniverse user')
zoo_pass = getpass.getpass('Enter your Zooniverse password')
project_n = getpass.getpass('Enter the number of the Zooniverse project')

In [ ]:
# Import required modules
import sys
sys.path.append('..')
import utils.t12_utils as t12

# Initiate the SQL database 
%run -i "../db_starter/starter.py" --user $zoo_user --passw $zoo_pass --project_n $project_n

### Step 1: Specify the Zooniverse workflow id and version of interest

*Note:  A manual export in Zooniverse is required to get the most up-to-date classifications here**

In [ ]:
# TODO Display a selectable list of workflow ids with their names and a list of versions of the workflow of interest
#workflow_id, workflow_version, subj_type = t13.get_workflows(zoo_user, zoo_pass)
    
# Temporary example objects
workflow_id = 17719
workflow_version = 0.0
subj_type = "clip"

# Retrieve classifications from the workflow of interest
total_df, class_df = t12.get_classifications(workflow_id, workflow_version, subj_type, zoo_user, zoo_pass, project_n)

### Step 2: Aggregate classifications received on the workflow of interest

In [ ]:
import utils.t12_utils as t12# TODO Display a selectable list of min number of users, agreement, 
#agg_user, min_users = t13.set_aggregation_parameters()

# Specify the agreement threshold required among cit scientists
agg_users = 0.8
min_users = 3

class_df1 = t12.aggregrate_classifications(class_df, subj_type, agg_users, min_users)

### Step 3: Summarise the number of classifications based on the agreement specified

### Step 4: Use the subject explorer widget to get more information about specific subjects and their aggregated classifications

In [ ]:
# Get all classified subjects from specified workflows
total_df, class_df = t13.get_exports(user, password)

clips_df = t13.process_clips(total_df, class_df, clips_workflow_id, clips_workflow_version)
frames_df = t13.process_frames(db_path, total_df, frames_workflow_id,
                               frames_workflow_version, 
                               "https://drive.google.com/file/d/1z72CqTtEBtqk6936H1YNrCjc5NRopF0g/view?usp=sharing")

t13.launch_viewer(total_df, clips_df, frames_df)

In [ ]:
# END